# Training of Group 2 Model

### Import the required modules

In [ ]:
import tensorflow as tf
from reader import AudioReader
from preprocessing import Padding, Normalization
from preprocessing import MFCC

### Define the Hyperparameters

In [2]:
PREPROCESSING_ARGS = {
    'sampling_rate': 16000,
    'frame_length_in_s': 0.016,
    'frame_step_in_s': 0.008,
    'num_mel_bins': 10,
    'lower_frequency': 40,
    'upper_frequency': 4000,
    'num_coefficients': 10
}

TRAINING_ARGS = {
    'batch_size': 20,
    'learning_rate': 0.01,
    'end_learning_rate': 1.e-6,
    'epochs': 22,
    'width_multiplier': 0.25
}

### Create train/val/test Datasets

In [3]:
train_ds = tf.data.Dataset.list_files(['/tmp/msc-train/up*', '/tmp/msc-train/down*'])
val_ds = tf.data.Dataset.list_files(['/tmp/msc-val/up*', '/tmp/msc-val/down*'])
test_ds = tf.data.Dataset.list_files(['/tmp/msc-test/up*', '/tmp/msc-test/down*'])

### Define the Data Pipeline

In [ ]:
audio_reader = AudioReader(tf.int16)
padding = Padding(PREPROCESSING_ARGS['sampling_rate'])
normalization = Normalization(tf.int16)
feature_processor = MFCC(**PREPROCESSING_ARGS)
feature_processor_fn = feature_processor.get_mfccs_and_label

LABELS = ['down', 'up']

def prepare_for_training(feature, label):
    feature = tf.expand_dims(feature, -1)
    label_id = tf.argmax(label == LABELS)

    return feature, label_id

train_ds = (train_ds
            .map(audio_reader.get_audio_and_label)
            .map(padding.pad)
            .map(normalization.normalize)
            .map(feature_processor_fn)
            .map(prepare_for_training)
            .batch(TRAINING_ARGS['batch_size'])
            .cache())
val_ds = (val_ds
            .map(audio_reader.get_audio_and_label)
            .map(padding.pad)
            .map(normalization.normalize)
            .map(feature_processor_fn)
            .map(prepare_for_training)
            .batch(TRAINING_ARGS['batch_size']))
test_ds = (test_ds
            .map(audio_reader.get_audio_and_label)
            .map(padding.pad)
            .map(normalization.normalize)
            .map(feature_processor_fn)
            .map(prepare_for_training)
            .batch(TRAINING_ARGS['batch_size']))

### Read a batch of data

In [ ]:
for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

### Create the Model and Apply Structured Pruning via Width Multiplier

In [6]:
wm = TRAINING_ARGS['width_multiplier']

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=int(256 * wm), kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=int(256 * wm), kernel_size=[1, 1], strides=[1, 1],
        use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=int(256 * wm), kernel_size=[1, 1], strides=[1, 1],
        use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
])

In [ ]:
model.summary()

### Create callbacks

In [8]:
# Learning Rate scheduler
linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=TRAINING_ARGS['learning_rate'],
    end_learning_rate=TRAINING_ARGS['end_learning_rate'],
    decay_steps=len(train_ds) * TRAINING_ARGS['epochs'],
)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(linear_decay)

# Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    mode='auto'
)

### Train the Model (with Callbacks)

- Modify the `learning_rate` argument of the `Adam` optimizer.
- Modify the `fit` method, specifying the `callbacks` argument.

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model.fit(
    train_ds, 
    epochs=TRAINING_ARGS['epochs'], 
    validation_data=val_ds, 
    callbacks=[lr_scheduler, early_stopping]
)

### Show the History

In [ ]:
history.history

### Evaluate the Model

In [ ]:
training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

test_loss, test_accuracy = model.evaluate(test_ds)

print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')

### Save the Model

In [ ]:
import os
import zipfile
from time import time
import tensorflow as tf

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)  # Assuming 'model' is a Keras model
tflite_model = converter.convert()

# Create a directory for the TFLite model zip file
tflite_model_dir = './tflite_models'
if not os.path.exists(tflite_model_dir):
    os.makedirs(tflite_model_dir)

# Define the path for the zip file and TFLite file inside it
tflite_zip_name = os.path.join(tflite_model_dir, 'model2.tflite.zip')

# Create a zip file and save the TFLite model in it with the name 'model2.tflite'
with zipfile.ZipFile(tflite_zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.writestr('model2.tflite', tflite_model)

# Output the path of the saved zip file
tflite_zip_name


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5f0c18e4-6e8e-4b7e-be1d-d8fab55c80d7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>